# Model 1: Tipus incidència 'No ha impartit classe a aquest grup'
## Part-of-speech method - Train

En este notebook se va ha intendar mejorar el modelo de detección de la incidencia 
'No ha impartit classe a aquest grup' mediante el uso de Part-of-Speech y Dependency Parsing.

Aquí se definirán las reglas para classificar las classes, observando los ejemplos.


In [1]:
import json
import os.path
import sys
import csv
import numpy as np
import pandas as pd
from colorama import Fore

In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

In [3]:
import spacy
from spacy.util import minibatch, compounding
from spacy import displacy
from spacy.matcher import Matcher

In [4]:
def load_data (language):
    
    if (debug >= 1):
        print ("LOAD_DATA")
        
    # Load data from file
    file = "comentaris_" + language + ".csv"
    data = pd.read_csv(pathdest + file)
    if (debug >= 1):
        print ("Original data:", data.shape[0])
    if (debug >= 2):
        display (data.sample(5))

    # Filter rows with 'TipusPregunta' = 'Profesor' and 'TipusIncidencia' = 'No ha impartit classe'
    # And select only column 'Comentari'
    data_profe = data[data.TipusPregunta == "P"]
    data_true = data_profe[data_profe.TipusIncidencia == "No ha impartit classe a aquest grup"][["Comentari"]]
    if (debug >= 1):
        print ("True cases:", data_true.shape[0])
    if (debug >= 2):
        display (data_true.sample(5))    
    
    # Group and count comments to delete duplicates
    data_counts = data_true['Comentari'].value_counts().index
    if (debug >= 1):
        print (data_counts[:5])
        print ()
    
#    display (data_counts.index)
    
#    data_group = data_true.groupby(['Comentari']).count()
#    display (data_group)
    
    #display (data_group.sort_values('counts'))
    
    return data_counts

In [20]:
def show_parts_of_speech(doc):
    
    print ('PARTS OF SPEECH:')    
    for sent in doc.sents:
        print ("Text: ", sent)
        print ()
        print ("TOKEN, TAG, POS, DEP, LEMMA, SHAPE, EXPLAIN")
        for token in sent:
            print (token, token.tag_, token.pos_, token.dep_, token.lemma_ )

        print ()
        print ('Dependency parsing:')
        for chunk in sent.noun_chunks:
            print (chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

        displacy.render(sent, style='dep', jupyter=True, options={'distance':90})


In [6]:
def train_matcher(nlp):
    #  Defining patterns

    print ("TRAIN MATCHER")
    matcher = Matcher(nlp.vocab, validate=True)

    adv_negs = ["no","tampoco"]
    verbs = ["tratar","ir","conocer","ver","saber","ser","aparecer","tener"]
    verbs_trans = ["realizar","dar","recibir","dictar"]
    noms_trans = ["clase","asignatura"]

    # Example: "No he tratado con él", "No sé quien es", "No ha aparecido"
    pattern = [{"POS": "ADV","LOWER":{"IN":adv_negs}}, {"OP": "*"}, 
               {"DEP":"ROOT","POS": "VERB", "LEMMA": {"IN": verbs}}]
    matcher.add("NoClasses1", None, pattern)

    # Example: "No ha realizado clases"
    pattern = [{"POS": "ADV","LOWER":{"IN":adv_negs}}, {"OP": "*"}, 
               {"DEP":"ROOT","POS": "VERB", "LEMMA": {"IN": verbs_trans}},
               {"OP": "*"}, {"POS":"NOUN","LEMMA":{"IN":noms_trans}}]
    matcher.add("NoClasses3", None, pattern)

    if (debug >= 1):
            print ("Patterns: ", len(matcher))
            
    return matcher

In [18]:
# Individual Rule-based matching

def test_individual_matching(doc):
    print ()
    print ("TEST INDIVIDUAL MATCHING:")

    for sent in doc.sents:
        print ("Sentence: ", sent)
    #    show_parts_of_speech(sent)

        #print ("Text: ", doc)
        matches = matcher(sent)
        if (len(matches) == 0):
            print ("No matches")
        else:
            for match_id, start, end in matches:
                string_id = nlp.vocab.strings[match_id]  # Get string representation
                span = sent[start:end]  # The matched span
                print(match_id, string_id, start, end, span.text)
        print ()
    

In [8]:
# General Rule-based matching

def test_global_matching(data_true):

    print ()
    print ("TEXT GLOBAL MATCHING:")

    for text in data_true:
        doc = nlp(text)
        
        test_individual_matching (doc)



In [10]:
pathori = "../data/original"
pathdest = "../data/preprocessed/"
pathmodel = "../data/models/"
debug = 1

In [11]:
languages = {"ca":"ca_fasttext_wiki", "es":"es_core_news_lg", "en":"en_core_web_sm" }

language = "es"
model = languages["es"]

data_true = load_data (language)

# Load the model form spacy
nlp = spacy.load(model)

LOAD_DATA
Original data: 1761
True cases: 38
Index(['No he tratado con él', 'No lo he tenido',
       'No se recibió clases con ella', 'No nos dio clases.',
       'no hemos recibido ni una clase por su parte en toda la cuarentena, ni suyas ni del resto de profesores. '],
      dtype='object')



In [21]:
doc = nlp(data_true[0])
show_parts_of_speech(doc)

doc2 = nlp("No lo sé porque no dictó clases. Debería controlar que el docente que sí lo hizo cumpla con el horario")
show_parts_of_speech(doc2)

PARTS OF SPEECH:
Text:  No he tratado con él

TOKEN, TAG, POS, DEP, LEMMA, SHAPE, EXPLAIN
No ADV__Polarity=Neg ADV advmod No
he AUX__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin AUX aux haber
tratado VERB__Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part VERB ROOT tratar
con ADP__AdpType=Prep ADP case con
él PRON__Case=Acc,Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs PRON obl él

Dependency parsing:
él él obl tratado


PARTS OF SPEECH:
Text:  No lo sé porque no dictó clases.

TOKEN, TAG, POS, DEP, LEMMA, SHAPE, EXPLAIN
No ADV__Polarity=Neg ADV advmod No
lo PRON__Case=Acc|Gender=Masc|Number=Sing|Person=3|PrepCase=Npr|PronType=Prs PRON obj el
sé VERB__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin VERB ROOT ser
porque SCONJ SCONJ mark porque
no ADV__Polarity=Neg ADV advmod no
dictó VERB__Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin VERB advcl dictar
clases NOUN__Gender=Fem|Number=Plur NOUN obj clase
. PUNCT__PunctType=Peri PUNCT punct .

Dependency parsing:
lo lo obj sé
clases clases obj dictó


Text:  Debería controlar que el docente que sí lo hizo cumpla con el horario

TOKEN, TAG, POS, DEP, LEMMA, SHAPE, EXPLAIN
Debería AUX__Mood=Cnd|Number=Sing|Person=3|VerbForm=Fin AUX aux Debería
controlar VERB__VerbForm=Inf VERB ROOT controlar
que SCONJ SCONJ mark que
el DET__Definite=Def|Gender=Masc|Number=Sing|PronType=Art DET det el
docente NOUN__Number=Sing NOUN nsubj docente
que PRON__PronType=Int,Rel SCONJ nsubj que
sí ADV ADV advcl sí
lo PRON__Case=Acc|Gender=Masc|Number=Sing|Person=3|PrepCase=Npr|PronType=Prs PRON obj el
hizo VERB__Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin VERB acl hacer
cumpla VERB__Mood=Sub|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin VERB ccomp cumplir
con ADP__AdpType=Prep ADP case con
el DET__Definite=Def|Gender=Masc|Number=Sing|PronType=Art DET det el
horario NOUN__Gender=Masc|Number=Sing NOUN obj horario

Dependency parsing:
el docente docente nsubj cumpla
lo lo obj hizo
el horario horario obj cumpla


In [13]:
matcher = train_matcher(nlp)

TRAIN MATCHER
Patterns:  2


In [19]:
test_individual_matching(doc2)


TEST INDIVIDUAL MATCHING:
Sentence:  No lo sé porque no dictó clases.
6239189525466108865 NoClasses1 0 3 No lo sé

Sentence:  Debería controlar que el docente que sí lo hizo cumpla con el horario
No matches



In [22]:


test_global_matching (data_true)


TEXT GLOBAL MATCHING:

TEST INDIVIDUAL MATCHING:
Sentence:  No he tratado con él
6239189525466108865 NoClasses1 0 3 No he tratado


TEST INDIVIDUAL MATCHING:
Sentence:  No lo he tenido
6239189525466108865 NoClasses1 0 4 No lo he tenido


TEST INDIVIDUAL MATCHING:
Sentence:  No se recibió clases con ella
17279635963034239788 NoClasses3 0 4 No se recibió clases


TEST INDIVIDUAL MATCHING:
Sentence:  No nos dio clases.
17279635963034239788 NoClasses3 0 4 No nos dio clases


TEST INDIVIDUAL MATCHING:
Sentence:  no hemos recibido ni una clase por su parte en toda la cuarentena, ni suyas ni del resto de profesores.
17279635963034239788 NoClasses3 0 6 no hemos recibido ni una clase


TEST INDIVIDUAL MATCHING:
Sentence:  No se sabe nada de el durante la cuarentena
6239189525466108865 NoClasses1 0 3 No se sabe


TEST INDIVIDUAL MATCHING:
Sentence:  No vimos clase con él
6239189525466108865 NoClasses1 0 2 No vimos


TEST INDIVIDUAL MATCHING:
Sentence:  No lo he tenido
6239189525466108865 NoClas